# Uber DataFrame
- This notebook combines multiple Market indicators into one massive DataFrame
- That DataFrame will be used for Machine Learning

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-Up" data-toc-modified-id="Set-Up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set Up</a></span></li><li><span><a href="#Fetch-Data" data-toc-modified-id="Fetch-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Fetch Data</a></span></li><li><span><a href="#Find-Common-Date-Span" data-toc-modified-id="Find-Common-Date-Span-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Find Common Date Span</a></span></li><li><span><a href="#Visualize-the-dependencies" data-toc-modified-id="Visualize-the-dependencies-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Visualize the dependencies</a></span></li><li><span><a href="#Divide-Training-and-Prediction-data-sets" data-toc-modified-id="Divide-Training-and-Prediction-data-sets-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Divide Training and Prediction data sets</a></span><ul class="toc-item"><li><span><a href="#Save-Datasets" data-toc-modified-id="Save-Datasets-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Save Datasets</a></span></li></ul></li></ul></div>

## Set Up

In [1]:
dir="../Data/processed/"

dataset_file_name = "uber.csv"

file1 = "^DJI.csv"
file2 = "united-states.ism-manufacturing-employment.csv"
file3 = "united-states.housing-starts-mm.csv"
file4 = "united-states.ism-manufacturing-pmi.csv"
file5 = "^VIX.csv"


include("../Julia/functions.jl") 
println()

## Fetch Data

In [2]:
df1 = fetch_dataset( file1 , "yyyy-mm-dd", dir )

using Statistics
describe(df1)

Fetched ../Data/processed/^DJI.csv, record count 13125


,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,Rata_Die,731232.0,724670,731232.0,737794,,,Int64
2,Date,,1985-01-29,,2021-01-04,13125,,Date
3,Original,10381.3,1251.21,10215.5,30218.3,,,Float64
4,Quantized,-47.623,-128.0,-49.0,127.0,,,Float64
5,Avg030,-47.6219,-128.0,-48.87,127.0,,,Float64
6,Avg060,-47.6193,-128.0,-48.52,124.55,,,Float64
7,Avg090,-47.6073,-127.97,-48.33,119.34,,,Float64
8,Avg120,-47.5903,-127.94,-48.41,116.07,,,Float64
9,Avg180,-47.5499,-127.66,-48.53,111.66,,,Float64


In [3]:
df2 = fetch_dataset( file2 , "yyyy-mm-dd", dir )

using Statistics
describe(df2)

Fetched ../Data/processed/united-states.ism-manufacturing-employment.csv, record count 2622


,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,Rata_Die,736484.0,735173,736484.0,737794,,,Int64
2,Date,,2013-11-01,,2021-01-04,2622,,Date
3,Original,52.1841,27.5,52.8,60.3,,,Float64
4,Quantized,63.9352,-128.0,69.0,127.0,,,Float64
5,Avg030,63.349,-128.0,69.185,127.0,,,Float64
6,Avg060,62.6054,-110.6,72.0,125.5,,,Float64
7,Avg090,61.7481,-79.27,71.925,124.68,,,Float64
8,Avg120,60.9625,-60.13,68.985,124.02,,,Float64
9,Avg180,59.7609,-36.78,68.44,114.71,,,Float64


In [4]:
df3 = fetch_dataset( file3 , "yyyy-mm-dd", dir )

using Statistics
describe(df3)

Fetched ../Data/processed/united-states.housing-starts-mm.csv, record count 1879


,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,Rata_Die,736885.0,735975,736885.0,737795,,,Int64
2,Date,,2016-01-12,,2021-01-05,1821,,Date
3,Original,-2.41197,-30.2,-2.5,25.5,,,Float64
4,Quantized,-0.604045,-128.0,-1.0,127.0,,,Float64
5,Avg030,-0.594125,-5.23,-0.73,3.27,,,Float64
6,Avg060,-0.580404,-4.63,-0.53,2.42,,,Float64
7,Avg090,-0.575168,-3.42,-0.53,1.62,,,Float64
8,Avg180,-0.552645,-2.04,-0.55,0.54,,,Float64
9,Avg365,-0.50099,-1.11,-0.57,0.0,,,Float64


In [5]:
df4 = fetch_dataset( file4 , "yyyy-mm-dd", dir )

using Statistics
describe(df3)

Fetched ../Data/processed/united-states.ism-manufacturing-pmi.csv, record count 5059


,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,Rata_Die,736885.0,735975,736885.0,737795,,,Int64
2,Date,,2016-01-12,,2021-01-05,1821,,Date
3,Original,-2.41197,-30.2,-2.5,25.5,,,Float64
4,Quantized,-0.604045,-128.0,-1.0,127.0,,,Float64
5,Avg030,-0.594125,-5.23,-0.73,3.27,,,Float64
6,Avg060,-0.580404,-4.63,-0.53,2.42,,,Float64
7,Avg090,-0.575168,-3.42,-0.53,1.62,,,Float64
8,Avg180,-0.552645,-2.04,-0.55,0.54,,,Float64
9,Avg365,-0.50099,-1.11,-0.57,0.0,,,Float64


In [6]:
df5 = fetch_dataset( file5 , "yyyy-mm-dd", dir )

using Statistics
describe(df3)

Fetched ../Data/processed/^VIX.csv, record count 11326


,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,Rata_Die,736885.0,735975,736885.0,737795,,,Int64
2,Date,,2016-01-12,,2021-01-05,1821,,Date
3,Original,-2.41197,-30.2,-2.5,25.5,,,Float64
4,Quantized,-0.604045,-128.0,-1.0,127.0,,,Float64
5,Avg030,-0.594125,-5.23,-0.73,3.27,,,Float64
6,Avg060,-0.580404,-4.63,-0.53,2.42,,,Float64
7,Avg090,-0.575168,-3.42,-0.53,1.62,,,Float64
8,Avg180,-0.552645,-2.04,-0.55,0.54,,,Float64
9,Avg365,-0.50099,-1.11,-0.57,0.0,,,Float64


## Find Common Date Span

Example:
- for all the DataFrames
- find lowest dates
- select the highest (common) of them

In [7]:
using Dates

function format_rata_die_to_us_date(d)
    return Dates.format( rata2datetime(d) , "u. d, yyyy")
end

format_rata_die_to_us_date (generic function with 1 method)

In [8]:
oldest_common_day = maximum([
         minimum(df1[:,1]) 
        ,minimum(df2[:,1]) 
        ,minimum(df3[:,1])
        ,minimum(df4[:,1])
        ,minimum(df5[:,1])
        ])


println("Oldest common day for df1 is ", format_rata_die_to_us_date( minimum(df1[:,1]))  )
println("Oldest common day for df2 is ", format_rata_die_to_us_date( minimum(df2[:,1]))  )
println("Oldest common day for df3 is ", format_rata_die_to_us_date( minimum(df3[:,1]))  )
println("Oldest common day for df4 is ", format_rata_die_to_us_date( minimum(df4[:,1]))  )
println("Oldest common day for df5 is ", format_rata_die_to_us_date( minimum(df5[:,1]))  )

println("Oldest common day is ", format_rata_die_to_us_date(oldest_common_day) )

Oldest common day for df1 is Jan. 29, 1985
Oldest common day for df2 is Nov. 1, 2013
Oldest common day for df3 is Jan. 12, 2016
Oldest common day for df4 is Mar. 1, 2007
Oldest common day for df5 is Jan. 2, 1990
Oldest common day is Jan. 12, 2016


In [9]:
newest_common_day = minimum([
          maximum(df1[:,1])
        , maximum(df2[:,1])
        , maximum(df3[:,1])
        , maximum(df4[:,1])
        , maximum(df5[:,1])
        ])

println("Newest common day for df1 is ", format_rata_die_to_us_date( maximum(df1[:,1]))  )
println("Newest common day for df2 is ", format_rata_die_to_us_date( maximum(df2[:,1]))  )
println("Newest common day for df3 is ", format_rata_die_to_us_date( maximum(df3[:,1]))  )
println("Newest common day for df4 is ", format_rata_die_to_us_date( maximum(df4[:,1]))  )
println("Newest common day for df5 is ", format_rata_die_to_us_date( maximum(df5[:,1]))  )

println("Newest common day is ", format_rata_die_to_us_date(newest_common_day) )

Newest common day for df1 is Jan. 4, 2021
Newest common day for df2 is Jan. 4, 2021
Newest common day for df3 is Jan. 5, 2021
Newest common day for df4 is Jan. 4, 2021
Newest common day for df5 is Jan. 4, 2021
Newest common day is Jan. 4, 2021


In [10]:
function trim_DataFrame(name::String, df::DataFrame, oldest_common_day::Int64,  newest_common_day::Int64)
    oldest_row = find_day(df, oldest_common_day)
    newest_row = find_day(df, newest_common_day)
    df = df[oldest_row:newest_row,:] # trimming
    println(name, " ", size(df)[1],"\n    ", names(df) )
    return df
end

df1 = trim_DataFrame("^DJI", df1, oldest_common_day, newest_common_day)
df2 = trim_DataFrame("mnfc_epmlt", df2, oldest_common_day, newest_common_day)
df3 = trim_DataFrame("hous_start", df3, oldest_common_day, newest_common_day)
df4 = trim_DataFrame("mnfc_PMI", df4, oldest_common_day, newest_common_day)
df5 = trim_DataFrame("^VIX", df5, oldest_common_day, newest_common_day)
println()

^DJI 1820
    [:Rata_Die, :Date, :Original, :Quantized, :Avg030, :Avg060, :Avg090, :Avg120, :Avg180, :Avg365]
mnfc_epmlt 1820
    [:Rata_Die, :Date, :Original, :Quantized, :Avg030, :Avg060, :Avg090, :Avg120, :Avg180, :Avg365]
hous_start 1878
    [:Rata_Die, :Date, :Original, :Quantized, :Avg030, :Avg060, :Avg090, :Avg180, :Avg365]
mnfc_PMI 1820
    [:Rata_Die, :Date, :Original, :Quantized, :Avg030, :Avg060, :Avg090, :Avg180, :Avg365]
^VIX 1820
    [:Rata_Die, :Date, :Original, :Quantized, :Avg030, :Avg060, :Avg090, :Avg120, :Avg180, :Avg365]



In [11]:
using DataFrames
uber_df = DataFrame(  Rata_Die              = df1[:,:Rata_Die] 
                     ,Date                  = df1[:,:Date] 
    
                     ,DJIA_Original         = df1[:,:Original] 
                     ,DJIA_Quantized        = df1[:,:Quantized] 
                     ,DJIA_Avg030           = df1[:,:Avg030] 
                     ,DJIA_Avg060           = df1[:,:Avg060] 
                     ,DJIA_Avg090           = df1[:,:Avg090] 
                     ,DJIA_Avg120           = df1[:,:Avg120] 
                     ,DJIA_Avg180          = df1[:,:Avg180] 
                     ,DJIA_Avg365          = df1[:,:Avg365] 
    
                     ,mnfc_epmlt_Original   = df2[:,:Original] 
                     ,mnfc_epmlt_Quantized  = df2[:,:Quantized]
                     ,mnfc_epmlt_Avg030     = df2[:,:Avg030]
                     ,mnfc_epmlt_Avg060     = df2[:,:Avg060]
                     ,mnfc_epmlt_Avg090     = df2[:,:Avg090] 
                     ,mnfc_epmlt_Avg120     = df1[:,:Avg120] 
                     ,mnfc_epmlt_Avg180     = df1[:,:Avg180]
                     ,mnfc_epmlt_Avg365    = df1[:,:Avg365] 
    
                     ,mnfc_PMI_Original     = df4[:,:Original] 
                     ,mnfc_PMI_Quantized    = df4[:,:Quantized] 
                     ,mnfc_PMI_Avg030       = df4[:,:Avg030]  
                     ,mnfc_PMI_Avg060       = df4[:,:Avg060]  
                     ,mnfc_PMI_Avg090       = df4[:,:Avg090] 
                    #,mnfc_PMI_Avg120       = df1[:,:Avg120] 
                     ,mnfc_PMI_Avg180       = df1[:,:Avg180] 
                     ,mnfc_PMI_Avg365       = df1[:,:Avg365] 
    
                     ,VIX_Original          = df5[:,:Original] 
                     ,VIX_Quantized         = df5[:,:Quantized] 
                     ,VIX_Avg030            = df5[:,:Avg030]  
                     ,VIX_Avg060            = df5[:,:Avg060]  
                     ,VIX_Avg090            = df5[:,:Avg090] 
                     ,VIX_Avg120            = df1[:,:Avg120] 
                     ,VIX_Avg180            = df1[:,:Avg180] 
                     ,VIX_Avg365            = df1[:,:Avg365] 

                   )
using Statistics
describe(uber_df)

,variable,mean,min,median,max,nunique,nmissing
,Symbol,Union…,Any,Union…,Any,Union…,Nothing
1,Rata_Die,736884.0,735975,736884.0,737794,,
2,Date,,2016-01-12,,2021-01-04,1820,
3,DJIA_Original,23769.2,15897.8,24750.2,30218.3,,
4,DJIA_Quantized,70.2346,1.0,79.0,127.0,,
5,DJIA_Avg030,69.3258,3.87,78.485,127.0,,
6,DJIA_Avg060,68.4263,5.73,78.95,124.55,,
7,DJIA_Avg090,67.5987,8.59,78.57,119.34,,
8,DJIA_Avg120,66.7864,10.54,79.455,116.07,,
9,DJIA_Avg180,65.1665,10.94,80.415,111.66,,


## Visualize the dependencies

In [12]:
columns = print_colunms(uber_df)
println()

1 Rata_Die
2 Date
3 DJIA_Original
4 DJIA_Quantized
5 DJIA_Avg030
6 DJIA_Avg060
7 DJIA_Avg090
8 DJIA_Avg120
9 DJIA_Avg180
10 DJIA_Avg365
11 mnfc_epmlt_Original
12 mnfc_epmlt_Quantized
13 mnfc_epmlt_Avg030
14 mnfc_epmlt_Avg060
15 mnfc_epmlt_Avg090
16 mnfc_epmlt_Avg120
17 mnfc_epmlt_Avg180
18 mnfc_epmlt_Avg365
19 mnfc_PMI_Original
20 mnfc_PMI_Quantized
21 mnfc_PMI_Avg030
22 mnfc_PMI_Avg060
23 mnfc_PMI_Avg090
24 mnfc_PMI_Avg180
25 mnfc_PMI_Avg365
26 VIX_Original
27 VIX_Quantized
28 VIX_Avg030
29 VIX_Avg060
30 VIX_Avg090
31 VIX_Avg120
32 VIX_Avg180
33 VIX_Avg365



In [ ]:
using Plots

record_count = size(uber_df)[1]
rows = 1:record_count
dates = format_dates( uber_df[rows,2] , "m/d/yy")

gr()
plot(          dates, # x-axis: dates
               [ uber_df[rows,6] uber_df[rows,14] uber_df[rows,22] uber_df[rows,29]         ], # y-axis
    label    = [      columns[6]      columns[14]      columns[22]      columns[29]      "" ],
    legend   =:bottom, 
              # :right, :left, :top, :bottom, :inside, :best, :legend, :topright, :topleft, :bottomleft, :bottomright
    xlabel   = "time",
    ylabel   = "indicators",
    size     = (980, 400), # width, height
    layout = (1, 1) # number of graphs: vertically, horizontally
    )

## Divide Training and Prediction data sets

In [ ]:
record_count = size(uber_df)[1]
today_rata = Dates.datetime2rata( today() )
today_id = find_day(uber_df, today_rata)

# I am putting overlap in the sets to see the behavior
df_training   = uber_df[1:today_id-5,:] 
df_prediction = uber_df[split_day-60:record_count,:]

println()

### Save Datasets

In [ ]:
save_dataset(df_training,   "uber_training.csv"  , "../Data/processed/");
save_dataset(df_prediction, "uber_prediction.csv", "../Data/processed/");